# Wissensaggregator Mittelalter und frühe Neuzeit

## Strukturdaten einlesen/aktualisieren

Inhalt

- [User anlegen](#User-anlegen)
- [Typen anlegen](#Typen-anlegen)
- [SKOS-Typen einlesen](#SKOS-Typen-einlesen)
- [Länder einlesen](#Länder-einlesen)
- [Verwaltungsebenen einlesen](#Verwaltungsebenen-einlesen)
- [Orte aus GeoNames einlesen](#Orte-aus-GeoNames-einlesen)
  - [Fremsprachliche Namen](#Fremdsprachliche-Namen)
  - [Deutsche Namen eintragen](#Deutsche-Namen-eintragen)
- [Quellen/Institutionen für externe IDs](#Quellen/Institutionen-für-externe-IDs)
- [Quellen/Referenzen](#Quellen/Referenzen)
- [Ämter](#Ämter)

Die Funktionen, welche Daten laden und umwandeln sind im Modul `WiagDataSetup` enthalten. Dieses verwendet seinerseits folgende Module:

- MySQL
- DataFrames
- JSON
- Infiltrator

Diese werden am besten vorab direkt in einem Julia-Terminal installiert.
``` julia
cd("C:\\Users\\Georg\\Documents\\projekte\\WiagDataSetup.jl")
Pkg.activate(".")
Pkg.add("MySQL")
Pkg.add("DataFrames")
Pkg.add("JSON")
Pkg.add("Infiltrator")
```

Pfad zum Modul `WiagDataSetup`

In [1]:
wds_path="../.."

"../.."

In [2]:
cd(wds_path)

In [3]:
using Pkg

In [4]:
Pkg.activate(".")

  Activating project at `C:\Users\georg\Documents\projekte\WiagDataSetup.jl`


Nur für die Entwicklung des Moduls relevant.

In [5]:
using Revise

Modul laden

In [6]:
using WiagDataSetup; Wds=WiagDataSetup

┌ Info: Precompiling WiagDataSetup [522c5ebb-a018-4020-8ed4-420cb1a9f084]
└ @ Base loading.jl:1423


WiagDataSetup

Verbinde die Datenbank.

Im Verlauf der Einleseschritte kann die folgende Fehlermeldung erscheinen:
*Commands out of sync; you can't run this command now*.
In diesem Fall diesen Befehl nochmal absetzen.

In [7]:
Wds.setDBWIAG(user="georg", db="wiag2")

Passwort für User georg: ········


MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag2")

Verzeichnis für Basisdaten, z.b. SKOS-Schemes, Länder

In [9]:
data_path="C:\\Users\\georg\\Documents\\projekte-doc\\WiagDataSetup"

"C:\\Users\\georg\\Documents\\projekte-doc\\WiagDataSetup"

## User anlegen

In [1]:
using CSV, DataFrames

In [20]:
user_file = joinpath(data_path, "csv", "user_wiag.csv")

"C:\\Users\\georg\\Documents\\projekte-doc\\WiagDataSetup\\csv\\user_wiag.csv"

In [37]:
df_user = CSV.read(user_file, DataFrame)

,id,note,user_id,nickname,givenname
,Int64,String?,Int64?,String7,String15?
1,1,"funktionaler User, z.B. für automatisierten Import",missing,wiag,missing
2,2,missing,11,bkroege,Bärbel
3,3,missing,12,cpopp,Christian
4,7,missing,13,ghertko,Georg


In [38]:
Wds.filltable!("user_wiag", df_user; clear_table=true)

┌ Info: Rows inserted: 4
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1171


4

## Typen anlegen

Alle Entitäten/items werden einem Typ zugeordnet, z.B. Diözese, Bischof, Domherr, Domherr aus GS. Der Typ bestimmt in welcher Tabelle Daten zu der Entität abgelegt werden. Es kann aber durchaus sein, dass auch unterschiedliche Typen in einer Tabelle versammelt sind, z.B. Bischöfe und Domherren in `person`.

In [9]:
using CSV, DataFrames

In [10]:
item_type_file = joinpath(data_path, "csv", "item_type.csv")

"C:\\Users\\georg\\Documents\\projekte-doc\\WiagDataSetup\\csv\\item_type.csv"

In [11]:
df_item_type = CSV.read(item_type_file, DataFrame)

,id,note,name
,Int64,String?,String15
1,1,Diözese des Alten Reiches,Diözese
2,2,missing,Kloster
3,3,missing,Domstift
4,4,Bischof oder Patriarch des Alten Reiches,Bischof
5,5,Domherr des Alten Reiches,Domherr
6,6,Domherr des Alten Reiches aus der Personendatenbank der Germania Sacra,Domherr GS
7,7,religiöser Orden,Orden
8,8,"Amt, Rolle",Amt


In [12]:
Wds.filltable!("item_type", df_item_type; clear_table=true)

┌ Info: Rows inserted: 8
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1184


8

## SKOS Typen einlesen
Für alle Entitäten, die einen SKOS-Typ besitzen, können SKOS-Beziehungen hinterlegt werden. Für diese Entitäten kann ein SKOS-Thesaurus aufgebaut werden und/oder es können Bezeichner *label* in verschiedenen Sprachen hinterlegt werden.

Quelle: händisch erstellt

In [9]:
using CSV, DataFrames

In [13]:
skos_scheme_file=joinpath(data_path, "csv", "skos_scheme.csv")

"C:\\Users\\georg\\Documents\\projekte-doc\\WiagDataSetup\\csv\\skos_scheme.csv"

In [14]:
df_skos_scheme = CSV.read(skos_scheme_file, DataFrame)

,id,note,name
,Int64,String63?,String31
1,1,missing,Diözese
2,2,kirchliche Ämter,Amt
3,3,religiöser Orden,Orden
4,4,"z.B. Kloster, Domstift",Einrichtung
5,5,z.B. Bundesländer; genutzt für Übersetzungen,Verwaltungseinheit


In [15]:
Wds.filltable!("skos_scheme", df_skos_scheme, clear_table = true)

┌ Info: Rows inserted: 5
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1171


5

## Länder einlesen
Die Quelle enthält im Wesentlichen Deutschland und seine Nachbarn. Die numerische ID ist der numerische ISO-Code.

Quelle: https://download.geonames.org/export/dump/countryInfo.txt

In [14]:
using CSV, MySQL, DataFrames

In [16]:
country_file=joinpath(data_path, "GeoNames", "countryInfo.txt")

"C:\\Users\\georg\\Documents\\projekte-doc\\WiagDataSetup\\GeoNames\\countryInfo.txt"

In [17]:
df_country = CSV.read(country_file, DataFrame);

┌ Warning: thread = 2 warning: only found 18 / 19 columns around data row: 276. Filling remaining columns with `missing`
└ @ CSV C:\Users\georg\.julia\packages\CSV\b4GfC\src\file.jl:622
┌ Warning: thread = 2 warning: only found 18 / 19 columns around data row: 276. Filling remaining columns with `missing`
└ @ CSV C:\Users\georg\.julia\packages\CSV\b4GfC\src\file.jl:622


In [18]:
first(df_country, 5)

,ISO,ISO3,ISO-Numeric,fips,Country,Capital
,String3,String3,Int64,String3?,String63,String31?
1,AD,AND,20,AN,Andorra,Andorra la Vella
2,AE,ARE,784,AE,United Arab Emirates,Abu Dhabi
3,AF,AFG,4,AF,Afghanistan,Kabul
4,AG,ATG,28,AC,Antigua and Barbuda,St. John's
5,AI,AIA,660,AV,Anguilla,The Valley


In [32]:
df_country[end-5:end, [Symbol("ISO-Numeric"), :Country]]

,ISO-Numeric,Country
,Int64,String63
1,175,Mayotte
2,710,South Africa
3,894,Zambia
4,716,Zimbabwe
5,891,Serbia and Montenegro
6,530,Netherlands Antilles


In [30]:
select_cols = [
    Symbol("ISO-Numeric") => :id,
    :Country => :name,
    :ISO => :country_code,
    :Capital => :capital
]

4-element Vector{Pair{Symbol, Symbol}}:
 Symbol("ISO-Numeric") => :id
              :Country => :name
                  :ISO => :country_code
              :Capital => :capital

In [31]:
df_country_db = select(df_country, select_cols...);

In [33]:
Wds.filltable!("country", df_country_db, clear_table = true)

┌ Info: Rows inserted: 252
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1171


252

GeoNames-IDs ergänzen

Quelle: `wiag_bundeslaender_normdaten_lhofman.xls` übertragen in `country.csv`

In [12]:
using MySQL, DataFrames, CSV

In [11]:
cy_file=joinpath(data_path, "csv", "country.csv")

"C:\\Users\\georg\\Documents\\projekte-doc\\WiagDataSetup\\csv\\country.csv"

In [17]:
df_cy=CSV.read(cy_file, DataFrame)

,id,note,name,country_code,country_code_3,geonames_id,comment
,Int64,Missing,String15,String3,String3,Int64,Missing
1,528,missing,Niederlande,NL,NLD,2750405,missing
2,56,missing,Belgien,BE,BEL,2802361,missing
3,250,missing,Frankreich,FR,FRA,3017382,missing
4,380,missing,Italien,IT,ITA,3175395,missing
5,756,missing,Schweiz,CH,CHE,2658434,missing
6,40,missing,Österreich,AT,AUT,2782113,missing
7,208,missing,Dänemark,DK,DNK,2623032,missing
8,616,missing,Polen,PL,POL,798544,missing
9,276,missing,Deutschland,DE,DEU,2921044,missing


In [21]:
sql="DROP TABLE IF EXISTS country_gn_id"
DBInterface.execute(Wds.dbwiag, sql);

In [22]:
sql="CREATE TEMPORARY TABLE country_gn_id (" *
"id INT," *
"country_code VARCHAR(31)," *
"country_code_3 VARCHAR(31)," *
"geonames_id INT)"
DBInterface.execute(Wds.dbwiag, sql);

In [23]:
Wds.filltable!("country_gn_id", df_cy[!, [:id, :country_code, :country_code_3, :geonames_id]])

┌ Info: Rows inserted: 13
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1171


13

In [26]:
sql="UPDATE country AS cy, (SELECT id, country_code, country_code_3, geonames_id FROM country_gn_id) as gn " *
"SET cy.country_code_3 = gn.country_code_3, cy.geonames_id = gn.geonames_id " *
"WHERE cy.id = gn.id"
DBInterface.execute(Wds.dbwiag, sql);

externe IDs ergänzen

In [10]:
cei_file=joinpath(data_path, "csv", "country_id_external.csv")

"C:\\Users\\georg\\Documents\\projekte-doc\\WiagDataSetup\\csv\\country_id_external.csv"

In [13]:
df_cei=CSV.read(cei_file, DataFrame, types=[Int, String, String, String, Int, String]);

In [62]:
size(df_cei)

(18, 6)

In [14]:
df_cei[1:7, :]

,authority_id,note,country_name,value,geonames_id,comment
,Int64,String?,String,String,Int64,String?
1,1,missing,Niederlande,4042203-3,2750405,missing
2,1,missing,Belgien,4005406-8,2802361,missing
3,1,missing,Frankreich,4018145-5,3017382,missing
4,1,missing,Italien,4027833-5,3175395,missing
5,1,missing,Dänemark,4010877-6,2623032,missing
6,1,missing,Polen,4046496-9,798544,missing
7,1,missing,Deutschland,4011882-4,2921044,missing


In [66]:
Wds.filltable!("place_id_external", df_cei[!, [:geonames_id, :authority_id, :value]])

┌ Info: Rows inserted: 18
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1171


18

externe IDs für Bundesländer ergänzen

Quelle: Quelle: `wiag_bundeslaender_normdaten_lhofman.xls` übertragen in `country_level_1_id_external.csv`

In [33]:
cei_l1_file=joinpath(data_path, "csv", "country_level_1_id_external.csv")

"C:\\Users\\georg\\Documents\\projekte-doc\\WiagDataSetup\\csv\\country_level_1_id_external.csv"

In [67]:
df_cei_l1=CSV.read(cei_l1_file, DataFrame, types=[Int, String, String, String, Int, String]);

In [68]:
df_cei_l1[1:7, :]

,authority_id,note,country_name,value,geonames_id,comment
,Int64,String?,String,String,Int64,String?
1,1,missing,Baden-Württemberg,4004176-1,2953481,missing
2,1,missing,Bayern,4005044-0,2951839,missing
3,1,missing,Berlin,4005728-8,2950157,missing
4,1,missing,Brandenburg,4007955-7,2945356,missing
5,1,missing,Bremen,4008135-7,2944387,missing
6,1,missing,Hamburg,4023118-5,2911297,missing
7,1,missing,Hessen,4024729-6,2905330,missing


In [69]:
Wds.filltable!("place_id_external", df_cei_l1[!, [:geonames_id, :authority_id, :value]])

┌ Info: Rows inserted: 32
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1171


32

Kontrolle

``` sql
SELECT name, country_code, authority_id, value 
FROM place_id_external AS pei
JOIN country_level_1 AS cl1 ON pei.geonames_id = cl1.geonames_id;
```

Es fehlen Hamburg, Bremen

## Verwaltungsebenen einlesen
Die Quelle enthält die oberste Verwaltungsebene nach dem Land selbst, also für Deutschland die Bundesländer, für Frankreich die Regionen. Die Quelle umfasst alle Daten in GeoNames und wird daher gefiltert.

In [381]:
using DataFrames; using CSV

In [382]:
cl1_file=joinpath(data_path, "Geonames", "admin1CodesASCII.txt")

"C:\\Users\\georg\\Documents\\projekte-doc\\WiagDataSetup\\Geonames\\admin1CodesASCII.txt"

In [383]:
df_cl1=CSV.read(cl1_file, DataFrame, header=["code", "name", "ascii_name", "geonames_id"]);

In [384]:
first(df_cl1, 7)

,code,name,ascii_name,geonames_id
,String15,String63,String63,Int64
1,AD.06,Sant Julià de Loria,Sant Julia de Loria,3039162
2,AD.05,Ordino,Ordino,3039676
3,AD.04,La Massana,La Massana,3040131
4,AD.03,Encamp,Encamp,3040684
5,AD.02,Canillo,Canillo,3041203
6,AD.07,Andorra la Vella,Andorra la Vella,3041566
7,AD.08,Escaldes-Engordany,Escaldes-Engordany,3338529


Länder Code extrahieren

In [385]:
get_country_code(code)=split(code::AbstractString, ".")[1]

get_country_code (generic function with 1 method)

In [386]:
df_cl1[!, :country_code] .= get_country_code.(df_cl1[!, :code]);

In [387]:
first(df_cl1, 7)

,code,name,ascii_name,geonames_id,country_code
,String15,String63,String63,Int64,SubStri…
1,AD.06,Sant Julià de Loria,Sant Julia de Loria,3039162,AD
2,AD.05,Ordino,Ordino,3039676,AD
3,AD.04,La Massana,La Massana,3040131,AD
4,AD.03,Encamp,Encamp,3040684,AD
5,AD.02,Canillo,Canillo,3041203,AD
6,AD.07,Andorra la Vella,Andorra la Vella,3041566,AD
7,AD.08,Escaldes-Engordany,Escaldes-Engordany,3338529,AD


Code der Verwaltungsebene extrahieren

In [388]:
get_admin1_code(code)=split(code::AbstractString, ".")[2]

get_admin1_code (generic function with 1 method)

In [389]:
df_cl1[!, :admin1_code] .= get_admin1_code.(df_cl1[!, :code]);

Länder auslesen

In [390]:
using MySQL

In [400]:
sql = "SELECT id as country_id, country_code FROM country " * 
"WHERE country_code in (SELECT distinct(country_code) FROM place)"
df_country_m = DBInterface.execute(Wds.dbwiag, sql) |> DataFrame;

In [404]:
delete!(df_country_m, 16)

,country_id,country_code
,Int32,String?
1,40,AT
2,56,BE
3,191,HR
4,203,CZ
5,208,DK
6,233,EE
7,250,FR
8,276,DE
9,380,IT


In [405]:
df_ccl1 = leftjoin(df_country_m, df_cl1, on = :country_code);

Index ergänzen

In [406]:
df_ccl1[!, :id] .= 1:size(df_ccl1, 1);

In [407]:
first(df_ccl1[!, [:id, :country_id, :country_code, :name]], 5)

,id,country_id,country_code,name
,Int64,Int32,String?,String63?
1,1,40,AT,Vienna
2,2,40,AT,Vorarlberg
3,3,40,AT,Tyrol
4,4,40,AT,Styria
5,5,40,AT,Salzburg


In [408]:
size(df_ccl1)

(246, 8)

In [409]:
import_cols = [:id, :country_id, :country_code, :admin1_code, :name, :ascii_name, :geonames_id]

7-element Vector{Symbol}:
 :id
 :country_id
 :country_code
 :admin1_code
 :name
 :ascii_name
 :geonames_id

In [410]:
names(df_ccl1)

8-element Vector{String}:
 "country_id"
 "country_code"
 "code"
 "name"
 "ascii_name"
 "geonames_id"
 "admin1_code"
 "id"

In [411]:
Wds.filltable!("country_level_1", select(df_ccl1, import_cols), clear_table=true)

┌ Info: Rows inserted: 246
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1171


246

## Orte aus GeoNames einlesen
Geonames stellt eine Sammlung aller Objekte (features) zur Verfügung. Alle Objekte eines Landes sind in jeweils einer Datei enthalten.
https://download.geonames.org/export/dump/

Lies Daten zu 
- Deutschland
- Nachbarländer
- Baltikum
- Kroatien
- Kaliningrad/Königsberg

Ländercodes: DE, NL, BE, FR, IT, CH, AT, DK, PL, LU, CZ, LI, LV, LT, HR, EE, RU

Pfad zu den Daten

In [35]:
gn_path = "C:\\Users\\georg\\Documents\\projekte-doc\\WiagDataSetup\\GeoNames"

"C:\\Users\\georg\\Documents\\projekte-doc\\WiagDataSetup\\GeoNames"

Wähle aus den Features in den Daten von GeoNames nur Orte (P) aus, sowie Klöster (MSTY) und Konvente (CVNT).
Siehe http://www.geonames.org/export/codes.html.

Filterfunktion definieren

In [46]:
function filter_places(feature_class, feature_code)
    return ((!ismissing(feature_class) && feature_class == "P")
            || (!ismissing(feature_code) && feature_code in ("MSTY", "CVNT")))
end

filter_places (generic function with 1 method)

Spaltennamen und Spaltentypen

Länder auslesen

In [22]:
using MySQL
using DataFrames
using CSV

In [56]:
df_country_m = DBInterface.execute(Wds.dbwiag, "SELECT id as country_id, country_code FROM country") |> DataFrame;

In [57]:
first(df_country_m, 5)

,country_id,country_code
,Int32,String?
1,0,XK
2,4,AF
3,8,AL
4,10,AQ
5,12,DZ


In [38]:
gn_header = ["geonames_id", "name", "asciiname", "alternatenames",
             "latitude", "longitude", "feature_class", "feature_code",
             "country_code", "cc2", "admin1_code", "admin2_code", "admin3_code", "admin4_code",
             "population", "elevation", "dem", "timezone", "modification_date"];

gn_types = [Int, String, String, String, Float64, Float64,
            String, String, String, String,
            String, String, String, String,
            Int, Int, Int, String, String];

In [39]:
place_cols = [:country_id, :name, :asciiname, :latitude, :longitude, 
        :feature_class, :feature_code, :country_code, :cc2, :admin1_code,
        :population, :elevation, :dem, :timezone, :modification_date, :geonames_id];

### Niederlande

In [66]:
gn_filename = joinpath(gn_path, "NL", "NL.txt")

"C:\\Users\\georg\\Documents\\projekte-doc\\WiagDataSetup\\GeoNames\\NL\\NL.txt"

In [67]:
gn_df = CSV.read(gn_filename, DataFrame, header=gn_header, types=gn_types);

filtern nach Art des Features

In [68]:
gn_dff = filter([:feature_class, :feature_code] => filter_places, gn_df);

In [69]:
size(gn_dff)

(7764, 19)

In [70]:
gn_dff = rightjoin(df_country_m, gn_dff, on = :country_code);

In [71]:
gn_dff[1:5, [:country_id, :name, :latitude, :longitude, :elevation, :dem, :population]]

,country_id,name,latitude,longitude,elevation,dem,population
,Int32?,String,Float64,Float64,Int64?,Int64,Int64
1,528,Den Oord,51.9708,5.27083,missing,3,0
2,528,Zwolle,52.5125,6.09444,missing,9,111805
3,528,Zwolle,52.0317,6.65556,missing,30,65
4,528,Zwingelspaan,51.66,4.49583,missing,-2,120
5,528,Zwinderen,52.7267,6.67639,missing,17,260


In [72]:
Wds.filltable!("place", gn_dff[!, place_cols], clear_table = false)

┌ Info: Rows inserted: 7764
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1171


7764

### Belgien

In [73]:
gn_filename = joinpath(gn_path, "BE", "BE.txt")

"C:\\Users\\georg\\Documents\\projekte-doc\\WiagDataSetup\\GeoNames\\BE\\BE.txt"

In [74]:
gn_df = CSV.read(gn_filename, DataFrame, header=gn_header, types=gn_types);

filtern nach Art des Features

In [75]:
gn_dff = filter([:feature_class, :feature_code] => filter_places, gn_df);

In [76]:
gn_dff[1:5, [:geonames_id, :name, :country_code, :population]]

,geonames_id,name,country_code,population
,Int64,String,String,Int64
1,2783081,Zwijndrecht,BE,18249
2,2783083,Zwijnaardse Dries,BE,0
3,2783084,Zwijnaarde,BE,7379
4,2783086,Zwijn,BE,0
5,2783087,Zwevezele,BE,5873


In [77]:
size(gn_dff)

(12671, 19)

Spalte für die Länder_ID einfügen

In [78]:
gn_dff = rightjoin(df_country_m, gn_dff, on = :country_code);

In [79]:
Wds.filltable!("place", gn_dff[!, place_cols], clear_table = false)

┌ Info: 10000
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1148
┌ Info: Rows inserted: 12671
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1171


12671

### Frankreich

In [80]:
gn_filename = joinpath(gn_path, "FR", "FR.txt")

"C:\\Users\\georg\\Documents\\projekte-doc\\WiagDataSetup\\GeoNames\\FR\\FR.txt"

In [81]:
gn_df = CSV.read(gn_filename, DataFrame, header=gn_header, types=gn_types);

filtern nach Art des Features

In [82]:
gn_dff = filter([:feature_class, :feature_code] => filter_places, gn_df);

In [83]:
gn_dff[1:5, [:geonames_id, :name, :country_code, :population]]

,geonames_id,name,country_code,population
,Int64,String,String,Int64
1,2967103,Peyrat-le-Château,FR,1140
2,2967107,Domecy-sur-le-Vault,FR,107
3,2967108,Blaye,FR,5277
4,2967109,Zuytpeene,FR,483
5,2967110,Zuydcoote,FR,1660


In [84]:
size(gn_dff)

(80885, 19)

Spalte für die Länder_ID einfügen

In [85]:
gn_dff = rightjoin(df_country_m, gn_dff, on = :country_code);

In [86]:
Wds.filltable!("place", gn_dff[!, place_cols], clear_table = false)

┌ Info: 10000
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1148
┌ Info: 20000
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1148
┌ Info: 30000
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1148
┌ Info: 40000
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1148
┌ Info: 50000
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1148
┌ Info: 60000
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1148
┌ Info: 70000
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1148
┌ Info: 80000
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1148
┌ Info: Rows inserted: 80885
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1171


80885

### Italien

In [87]:
gn_filename = joinpath(gn_path, "IT", "IT.txt")

"C:\\Users\\georg\\Documents\\projekte-doc\\WiagDataSetup\\GeoNames\\IT\\IT.txt"

In [88]:
gn_df = CSV.read(gn_filename, DataFrame, header=gn_header, types=gn_types);

filtern nach Art des Features

In [89]:
gn_dff = filter([:feature_class, :feature_code] => filter_places, gn_df);

In [90]:
gn_dff[1:5, [:geonames_id, :name, :country_code, :population]]

,geonames_id,name,country_code,population
,Int64,String,String,Int64
1,781059,Colognole,IT,128
2,781060,Casale Sant'Antonio,IT,59
3,2522676,Zungti,IT,0
4,2522677,Zumpano,IT,343
5,2522679,Zona,IT,0


In [91]:
size(gn_dff)

(62844, 19)

Spalte für die Länder_ID einfügen

In [92]:
gn_dff = rightjoin(df_country_m, gn_dff, on = :country_code);

In [93]:
Wds.filltable!("place", gn_dff[!, place_cols], clear_table = false)

┌ Info: 10000
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1148
┌ Info: 20000
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1148
┌ Info: 30000
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1148
┌ Info: 40000
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1148
┌ Info: 50000
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1148
┌ Info: 60000
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1148
┌ Info: Rows inserted: 62844
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1171


62844

### Schweiz

In [94]:
gn_filename = joinpath(gn_path, "CH", "CH.txt")

"C:\\Users\\georg\\Documents\\projekte-doc\\WiagDataSetup\\GeoNames\\CH\\CH.txt"

In [95]:
gn_df = CSV.read(gn_filename, DataFrame, header=gn_header, types=gn_types);

filtern nach Art des Features

In [96]:
gn_dff = filter([:feature_class, :feature_code] => filter_places, gn_df);

In [97]:
gn_dff[1:5, [:geonames_id, :name, :country_code, :population]]

,geonames_id,name,country_code,population
,Int64,String,String,Int64
1,2657885,Zwischbergen,CH,127
2,2657886,Zwingen,CH,2162
3,2657887,Zweisimmen,CH,2813
4,2657888,Zweilütschinen,CH,0
5,2657889,Zuzwil,CH,4226


In [98]:
size(gn_dff)

(13265, 19)

Spalte für die Länder_ID einfügen

In [99]:
gn_dff = rightjoin(df_country_m, gn_dff, on = :country_code);

In [100]:
Wds.filltable!("place", gn_dff[!, place_cols], clear_table = false)

┌ Info: 10000
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1148
┌ Info: Rows inserted: 13265
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1171


13265

### Österreich

In [101]:
gn_filename = joinpath(gn_path, "AT", "AT.txt")

"C:\\Users\\georg\\Documents\\projekte-doc\\WiagDataSetup\\GeoNames\\AT\\AT.txt"

In [102]:
gn_df = CSV.read(gn_filename, DataFrame, header=gn_header, types=gn_types);

filtern nach Art des Features

In [103]:
gn_dff = filter([:feature_class, :feature_code] => filter_places, gn_df);

In [104]:
gn_dff[1:5, [:geonames_id, :name, :country_code, :population]]

,geonames_id,name,country_code,population
,Int64,String,String,Int64
1,2598250,Mooshöhe,AT,0
2,2598294,Muttling,AT,0
3,2598296,Kleiner,AT,0
4,2598303,Dörfl,AT,0
5,2598321,Zeitschen,AT,0


In [105]:
size(gn_dff)

(20924, 19)

Spalte für die Länder_ID einfügen

In [106]:
gn_dff = rightjoin(df_country_m, gn_dff, on = :country_code);

In [107]:
Wds.filltable!("place", gn_dff[!, place_cols], clear_table = false)

┌ Info: 10000
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1148
┌ Info: 20000
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1148
┌ Info: Rows inserted: 20924
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1171


20924

### Dänemark

In [108]:
gn_filename = joinpath(gn_path, "DK", "DK.txt")

"C:\\Users\\georg\\Documents\\projekte-doc\\WiagDataSetup\\GeoNames\\DK\\DK.txt"

In [109]:
gn_df = CSV.read(gn_filename, DataFrame, header=gn_header, types=gn_types);

filtern nach Art des Features

In [110]:
gn_dff = filter([:feature_class, :feature_code] => filter_places, gn_df);

In [111]:
gn_dff[1:5, [:geonames_id, :name, :country_code, :population]]

,geonames_id,name,country_code,population
,Int64,String,String,Int64
1,2609911,Yttrup,DK,0
2,2609915,Yppenbjerg,DK,0
3,2609916,Ypnested,DK,0
4,2609922,Yding,DK,0
5,2609926,Yderik,DK,0


In [112]:
size(gn_dff)

(7261, 19)

Spalte für die Länder_ID einfügen

In [113]:
gn_dff = rightjoin(df_country_m, gn_dff, on = :country_code);

In [114]:
Wds.filltable!("place", gn_dff[!, place_cols], clear_table = false)

┌ Info: Rows inserted: 7261
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1171


7261

### Polen

In [115]:
gn_filename = joinpath(gn_path, "PL", "PL.txt")

"C:\\Users\\georg\\Documents\\projekte-doc\\WiagDataSetup\\GeoNames\\PL\\PL.txt"

In [116]:
gn_df = CSV.read(gn_filename, DataFrame, header=gn_header, types=gn_types);

filtern nach Art des Features

In [117]:
gn_dff = filter([:feature_class, :feature_code] => filter_places, gn_df);

In [118]:
gn_dff[1:5, [:geonames_id, :name, :country_code, :population]]

,geonames_id,name,country_code,population
,Int64,String,String,Int64
1,462259,Zodenen,PL,0
2,620115,Włodawka,PL,0
3,688812,Vul’ka Ugruska,PL,0
4,696099,Pshedmes’tse Vel’ke,PL,0
5,698000,Paportno,PL,0


In [119]:
size(gn_dff)

(47411, 19)

Spalte für die Länder_ID einfügen

In [120]:
gn_dff = rightjoin(df_country_m, gn_dff, on = :country_code);

In [121]:
Wds.filltable!("place", gn_dff[!, place_cols], clear_table = false)

┌ Info: 10000
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1148
┌ Info: 20000
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1148
┌ Info: 30000
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1148
┌ Info: 40000
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1148
┌ Info: Rows inserted: 47411
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1171


47411

### Deutschland

In [58]:
gn_filename = joinpath(gn_path, "DE", "DE.txt")

"C:\\Users\\georg\\Documents\\projekte-doc\\WiagDataSetup\\GeoNames\\DE\\DE.txt"

In [59]:
gn_df = CSV.read(gn_filename, DataFrame, header=gn_header, types=gn_types);

filtern nach Art des Features

In [60]:
gn_dff = filter([:feature_class, :feature_code] => filter_places, gn_df);

In [61]:
gn_dff[1:5, [:geonames_id, :name, :country_code, :population]]

,geonames_id,name,country_code,population
,Int64,String,String,Int64
1,2657946,Wyhlen,DE,0
2,2803468,Zyfflich,DE,0
3,2803469,Zwuschwitz,DE,0
4,2803470,Zwötzen,DE,0
5,2803472,Zwotental,DE,0


In [62]:
size(gn_dff)

(79810, 19)

Spalte für die Länder_ID einfügen

In [63]:
gn_dff = rightjoin(df_country_m, gn_dff, on = :country_code);

In [64]:
gn_dff[1:5, :]

,country_id,country_code,geonames_id,name,asciiname,alternatenames
,Int32?,String,Int64,String,String,String?
1,276,DE,2657946,Wyhlen,Wyhlen,Wyhlen
2,276,DE,2803468,Zyfflich,Zyfflich,Zyfflich
3,276,DE,2803469,Zwuschwitz,Zwuschwitz,missing
4,276,DE,2803470,Zwötzen,Zwoetzen,"Gera-Zwotzen,Gera-Zwötzen,Zwotzen,Zwötzen"
5,276,DE,2803472,Zwotental,Zwotental,missing


In [65]:
Wds.filltable!("place", gn_dff[!, place_cols], clear_table = false)

┌ Info: 10000
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1148
┌ Info: 20000
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1148
┌ Info: 30000
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1148
┌ Info: 40000
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1148
┌ Info: 50000
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1148
┌ Info: 60000
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1148
┌ Info: 70000
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1148
┌ Info: Rows inserted: 79810
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1171


79810

### Luxemburg

In [122]:
gn_filename = joinpath(gn_path, "LU", "LU.txt")

"C:\\Users\\georg\\Documents\\projekte-doc\\WiagDataSetup\\GeoNames\\LU\\LU.txt"

In [123]:
gn_df = CSV.read(gn_filename, DataFrame, header=gn_header, types=gn_types);

filtern nach Art des Features

In [124]:
gn_dff = filter([:feature_class, :feature_code] => filter_places, gn_df);

In [125]:
gn_dff[1:5, [:geonames_id, :name, :country_code, :population]]

,geonames_id,name,country_code,population
,Int64,String,String,Int64
1,2959958,Zittig,LU,38
2,2959959,Wormeldange,LU,786
3,2959960,Wues,LU,0
4,2959961,Wolwelange,LU,358
5,2959964,Wolpert,LU,0


In [126]:
size(gn_dff)

(692, 19)

Spalte für die Länder_ID einfügen

In [127]:
gn_dff = rightjoin(df_country_m, gn_dff, on = :country_code);

In [128]:
Wds.filltable!("place", gn_dff[!, place_cols], clear_table = false)

┌ Info: Rows inserted: 692
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1171


692

### Tschechien

In [129]:
gn_filename = joinpath(gn_path, "CZ", "CZ.txt")

"C:\\Users\\georg\\Documents\\projekte-doc\\WiagDataSetup\\GeoNames\\CZ\\CZ.txt"

In [130]:
gn_df = CSV.read(gn_filename, DataFrame, header=gn_header, types=gn_types);

filtern nach Art des Features

In [131]:
gn_dff = filter([:feature_class, :feature_code] => filter_places, gn_df);

In [132]:
gn_dff[1:5, [:geonames_id, :name, :country_code, :population]]

,geonames_id,name,country_code,population
,Int64,String,String,Int64
1,3059873,Hrčava,CZ,0
2,3061283,Janská Hut,CZ,0
3,3061284,Dvůr Králové nad Labem,CZ,16150
4,3061285,Zvůle,CZ,0
5,3061286,Zvotoky,CZ,69


In [133]:
size(gn_dff)

(16445, 19)

Spalte für die Länder_ID einfügen

In [134]:
gn_dff = rightjoin(df_country_m, gn_dff, on = :country_code);

In [135]:
Wds.filltable!("place", gn_dff[!, place_cols], clear_table = false)

┌ Info: 10000
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1148
┌ Info: Rows inserted: 16445
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1171


16445

### Liechtenstein

In [136]:
gn_filename = joinpath(gn_path, "LI", "LI.txt")

"C:\\Users\\georg\\Documents\\projekte-doc\\WiagDataSetup\\GeoNames\\LI\\LI.txt"

In [137]:
gn_df = CSV.read(gn_filename, DataFrame, header=gn_header, types=gn_types);

filtern nach Art des Features

In [138]:
gn_dff = filter([:feature_class, :feature_code] => filter_places, gn_df);

In [139]:
gn_dff[1:5, [:geonames_id, :name, :country_code, :population]]

,geonames_id,name,country_code,population
,Int64,String,String,Int64
1,3042029,Vorderer Schellenberg,LI,0
2,3042030,Vaduz,LI,5197
3,3042033,Triesenberg,LI,2689
4,3042035,Triesen,LI,4701
5,3042037,Schellenberg,LI,1004


In [140]:
size(gn_dff)

(361, 19)

Spalte für die Länder_ID einfügen

In [141]:
gn_dff = rightjoin(df_country_m, gn_dff, on = :country_code);

In [142]:
Wds.filltable!("place", gn_dff[!, place_cols], clear_table = false)

┌ Info: Rows inserted: 361
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1171


361

### Estland

Die erste Zeile der Daten kann nicht gelesen werden. Sie wird manuell aus den Quelldaten gelöscht. Das ist unerheblich, weil es sich nicht um ein relevantes Feature handelt.

In [168]:
gn_filename = joinpath(gn_path, "EE", "EE-x1.txt")

"C:\\Users\\georg\\Documents\\projekte-doc\\WiagDataSetup\\GeoNames\\EE\\EE-x1.txt"

In [169]:
gn_df = CSV.read(gn_filename, DataFrame, header=gn_header, types=gn_types);

filtern nach Art des Features

In [170]:
gn_dff = filter([:feature_class, :feature_code] => filter_places, gn_df);

In [171]:
size(gn_dff)

(7059, 19)

In [172]:
gn_dff[1:5, [:geonames_id, :name, :country_code, :population]]

,geonames_id,name,country_code,population
,Int64,String,String,Int64
1,456463,Puijas,EE,0
2,587436,Nehatu,EE,0
3,587437,Mereküla,EE,0
4,587438,Krundiküla,EE,0
5,587439,Järve,EE,0


In [173]:
size(gn_dff)

(7059, 19)

Spalte für die Länder_ID einfügen

In [174]:
gn_dff = rightjoin(df_country_m, gn_dff, on = :country_code);

In [175]:
Wds.filltable!("place", gn_dff[!, place_cols], clear_table = false)

┌ Info: Rows inserted: 7059
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1171


7059

### Lettland

In [143]:
gn_filename = joinpath(gn_path, "LV", "LV.txt")

"C:\\Users\\georg\\Documents\\projekte-doc\\WiagDataSetup\\GeoNames\\LV\\LV.txt"

In [144]:
gn_df = CSV.read(gn_filename, DataFrame, header=gn_header, types=gn_types);

filtern nach Art des Features

In [145]:
gn_dff = filter([:feature_class, :feature_code] => filter_places, gn_df);

In [146]:
gn_dff[1:5, [:geonames_id, :name, :country_code, :population]]

,geonames_id,name,country_code,population
,Int64,String,String,Int64
1,453754,Valmiera,LV,26963
2,453756,Jaunzemji,LV,0
3,453758,Grīžukrogs,LV,0
4,453759,Čolēni,LV,0
5,453764,Zvirgzdene,LV,0


In [147]:
size(gn_dff)

(4091, 19)

Spalte für die Länder_ID einfügen

In [148]:
gn_dff = rightjoin(df_country_m, gn_dff, on = :country_code);

In [149]:
Wds.filltable!("place", gn_dff[!, place_cols], clear_table = false)

┌ Info: Rows inserted: 4091
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1171


4091

### Litauen

In [150]:
gn_filename = joinpath(gn_path, "LT", "LT.txt")

"C:\\Users\\georg\\Documents\\projekte-doc\\WiagDataSetup\\GeoNames\\LT\\LT.txt"

In [151]:
gn_df = CSV.read(gn_filename, DataFrame, header=gn_header, types=gn_types);

filtern nach Art des Features

In [152]:
gn_dff = filter([:feature_class, :feature_code] => filter_places, gn_df);

In [153]:
gn_dff[1:5, [:geonames_id, :name, :country_code, :population]]

,geonames_id,name,country_code,population
,Int64,String,String,Int64
1,592647,Bileišiai,LT,0
2,592648,Abakai,LT,0
3,592650,Zypliai,LT,0
4,592651,Žyniai,LT,0
5,592652,Žyniai,LT,0


In [154]:
size(gn_dff)

(19854, 19)

Spalte für die Länder_ID einfügen

In [155]:
gn_dff = rightjoin(df_country_m, gn_dff, on = :country_code);

In [156]:
Wds.filltable!("place", gn_dff[!, place_cols], clear_table = false)

┌ Info: 10000
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1148
┌ Info: Rows inserted: 19854
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1171


19854

### Kroatien

In [157]:
gn_filename = joinpath(gn_path, "HR", "HR.txt")

"C:\\Users\\georg\\Documents\\projekte-doc\\WiagDataSetup\\GeoNames\\HR\\HR.txt"

In [158]:
gn_df = CSV.read(gn_filename, DataFrame, header=gn_header, types=gn_types);

filtern nach Art des Features

In [159]:
gn_dff = filter([:feature_class, :feature_code] => filter_places, gn_df);

In [160]:
gn_dff[1:5, [:geonames_id, :name, :country_code, :population]]

,geonames_id,name,country_code,population
,Int64,String,String,Int64
1,3186233,Vranjic,HR,1110
2,3186247,Zvonik,HR,87
3,3186248,Zvoneća,HR,0
4,3186263,Zverinac,HR,43
5,3186265,Zvekovac,HR,193


In [161]:
size(gn_dff)

(9902, 19)

Spalte für die Länder_ID einfügen

In [162]:
gn_dff = rightjoin(df_country_m, gn_dff, on = :country_code);

In [163]:
Wds.filltable!("place", gn_dff[!, place_cols], clear_table = false)

┌ Info: Rows inserted: 9902
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1171


9902

### Königsberg - Russland

In [176]:
gn_filename = joinpath(gn_path, "RU", "RU.txt")

"C:\\Users\\georg\\Documents\\projekte-doc\\WiagDataSetup\\GeoNames\\RU\\RU.txt"

In [177]:
gn_df = CSV.read(gn_filename, DataFrame, header=gn_header, types=gn_types);

filtern nach Art des Features

In [178]:
gn_dff = filter([:feature_class, :feature_code] => filter_places, gn_df);

In [179]:
gn_dff[1:5, [:geonames_id, :name, :country_code, :population]]

,geonames_id,name,country_code,population
,Int64,String,String,Int64
1,451747,Zyabrikovo,RU,0
2,451748,Znamenka,RU,0
3,451749,Zhukovo,RU,0
4,451750,Zhitovo,RU,0
5,451751,Zhitnikovo,RU,0


In [180]:
size(gn_dff)

(201312, 19)

Extrahiere Königsberg

In [181]:
gn_dff_kb = filter(:name => isequal("Kaliningrad"), gn_dff)

,geonames_id,name,asciiname,alternatenames
,Int64,String,String?,String?
1,554234,Kaliningrad,Kaliningrad,"Caliningrado,Calininopolis,KGD,Kalinin'nkrant,Kaliningrad,Kaliningrada,Kaliningradas,Kaliningrado,Kaliningradum,Kaliningrau,Kaliningráu,Kalininqrad,Kalinjingrad,Kalinyingrad,Kalinyingrád,Kalińingrad,Kalíníngrad,Karaliaucios,Karaliaucius,Karaliaučios,Karaliaučius,Kaļiņingrada,Kenisberg,Koenigsbarg,Koenigsberg,Koenigsberg in Preussen,Korigsberg,Krolewiec,Królewiec,Kënisberg,Königsbarg,Königsberg,Königsberg in Preußen,Körigsberg,jia li ning ge lei,kalininagrada,kalliningeuladeu,kalynynghrad,kalynyngrad,kariningurado,qlynyngrd,Καλίνινγκραντ,Калининград,Калињинград,Калінінград,Կալինինգրադ,קלינינגרד,كالينينغراد,کالیننگراڈ,کالینینگراد,کیلننگراڈ,कालिनिनग्राद,ಕಲಿನಿನ್\u200dಗ್ರಾಡ್,კალინინგრადი,カリーニングラード,加里寧格勒,칼리닌그라드"


Spalte für die Länder_ID einfügen

In [184]:
gn_dff_kb = rightjoin(df_country_m, gn_dff_kb, on = :country_code);

In [185]:
Wds.filltable!("place", gn_dff_kb[!, place_cols], clear_table = false)

┌ Info: Rows inserted: 1
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1171


1

In [186]:
DBInterface.execute(Wds.dbwiag, "SELECT COUNT(*) FROM place") |> DataFrame

,COUNT(*)
,Int64
1,391240


### Fremdsprachliche Namen

In [9]:
using CSV, DataFrames, MySQL

In [17]:
gnl_path = "C:\\Users\\georg\\Documents\\projekte-doc\\WiagDataSetup\\GeoNames\\alternatenames"

"C:\\Users\\georg\\Documents\\projekte-doc\\WiagDataSetup\\GeoNames\\alternatenames"

In [56]:
gnl_header = ["id", "geonames_id", "lang", "label", 
              "is_preferred", "isShort", "isColloquial", "is_historic", "from", "to"];

gnl_types = [Int, Int, String, String,
             Int, Int, Int, Int, String, String];

In [15]:
lang_codes = ["la", "fr", "cz", "de", "pl", "en", "nl", "it"];

In [48]:
filter_lang(lc) = !ismissing(lc) && lc in lang_codes

filter_lang (generic function with 1 method)

In [24]:
country_codes = ["DE", "NL", "BE", "FR", "IT", "CH", "AT", "DK",
    "PL", "LU", "CZ", "LI", "LV", "LT", "HR", "EE", "RU"];

### Deutschland

In [229]:
cc = "DE"

"DE"

In [230]:
gnl_filename = joinpath(gnl_path, cc, cc * ".txt")

"C:\\Users\\georg\\Documents\\projekte-doc\\WiagDataSetup\\GeoNames\\alternatenames\\DE\\DE.txt"

In [231]:
gnl_df = CSV.read(gnl_filename, DataFrame, header=gnl_header, types=gnl_types);

In [232]:
size(gnl_df)

(217238, 10)

In [233]:
gnl_df = filter(:lang => filter_lang, gnl_df);

In [234]:
size(gnl_df)

(29410, 10)

Bundesländer/Kantone/Regionen einlesen, um relevante Namen zu übernehmen

In [235]:
sql = "SELECT name, geonames_id " *
      "FROM country_level_1 WHERE country_code = '" * cc * "'";
cl_df = DBInterface.execute(Wds.dbwiag, sql) |> DataFrame;

In [236]:
gnl_cl_df = innerjoin(gnl_df, cl_df, on = :geonames_id);

In [237]:
size(gnl_cl_df)

(117, 11)

In [239]:
Wds.filltable!("place_label", select(gnl_cl_df, Not([:isShort, :isColloquial, :from, :to, :name])))

┌ Info: Rows inserted: 117
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1171


117

Orte einlesen, um nur relevante Namen zu übernehmen

In [40]:
sql = "SELECT name, geonames_id " *
      "FROM place WHERE country_code = '" * cc * "'";
p_df = DBInterface.execute(Wds.dbwiag, sql) |> DataFrame;

In [43]:
size(p_df)

(79810, 2)

In [61]:
gnl_p_df = innerjoin(gnl_df, p_df, on = :geonames_id);

In [62]:
Wds.filltable!("place_label", select(gnl_p_df, Not([:isShort, :isColloquial, :from, :to, :name])))

┌ Info: Rows inserted: 4364
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1171


4364

### Niederlande

In [240]:
cc = "NL"

"NL"

In [241]:
gnl_filename = joinpath(gnl_path, cc, cc * ".txt")

"C:\\Users\\georg\\Documents\\projekte-doc\\WiagDataSetup\\GeoNames\\alternatenames\\NL\\NL.txt"

In [242]:
gnl_df = CSV.read(gnl_filename, DataFrame, header=gnl_header, types=gnl_types);

In [243]:
size(gnl_df)

(28157, 10)

In [244]:
gnl_df = filter(:lang => filter_lang, gnl_df);

In [245]:
size(gnl_df)

(3262, 10)

Bundesländer/Kantone/Regionen einlesen, um relevante Namen zu übernehmen

In [246]:
sql = "SELECT name, geonames_id " *
      "FROM country_level_1 WHERE country_code = '" * cc * "'";
cl_df = DBInterface.execute(Wds.dbwiag, sql) |> DataFrame;

In [247]:
gnl_cl_df = innerjoin(gnl_df, cl_df, on = :geonames_id);

In [249]:
size(gnl_cl_df)

(72, 11)

In [250]:
Wds.filltable!("place_label", select(gnl_cl_df, Not([:isShort, :isColloquial, :from, :to, :name])))

┌ Info: Rows inserted: 72
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1171


72

Orte einlesen, um nur relevante Namen zu übernehmen

In [69]:
sql = "SELECT name, geonames_id " *
      "FROM place WHERE country_code = '" * cc * "'";
p_df = DBInterface.execute(Wds.dbwiag, sql) |> DataFrame;

In [70]:
size(p_df)

(7764, 2)

In [71]:
gnl_p_df = innerjoin(gnl_df, p_df, on = :geonames_id);

In [72]:
Wds.filltable!("place_label", select(gnl_p_df, Not([:isShort, :isColloquial, :from, :to, :name])))

┌ Info: Rows inserted: 1848
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1171


1848

### Belgien

In [251]:
cc = "BE"

"BE"

In [252]:
gnl_filename = joinpath(gnl_path, cc, cc * ".txt")

"C:\\Users\\georg\\Documents\\projekte-doc\\WiagDataSetup\\GeoNames\\alternatenames\\BE\\BE.txt"

In [253]:
gnl_df = CSV.read(gnl_filename, DataFrame, header=gnl_header, types=gnl_types);

In [254]:
size(gnl_df)

(30448, 10)

In [255]:
gnl_df = filter(:lang => filter_lang, gnl_df);

In [256]:
size(gnl_df)

(2825, 10)

Bundesländer/Kantone/Regionen einlesen, um relevante Namen zu übernehmen

In [257]:
sql = "SELECT name, geonames_id " *
      "FROM country_level_1 WHERE country_code = '" * cc * "'";
cl_df = DBInterface.execute(Wds.dbwiag, sql) |> DataFrame;

In [258]:
gnl_cl_df = innerjoin(gnl_df, cl_df, on = :geonames_id);

In [259]:
size(gnl_cl_df)

(26, 11)

In [260]:
Wds.filltable!("place_label", select(gnl_cl_df, Not([:isShort, :isColloquial, :from, :to, :name])))

┌ Info: Rows inserted: 26
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1171


26

Orte einlesen, um nur relevante Namen zu übernehmen

In [79]:
sql = "SELECT name, geonames_id " *
      "FROM place WHERE country_code = '" * cc * "'";
p_df = DBInterface.execute(Wds.dbwiag, sql) |> DataFrame;

In [80]:
size(p_df)

(12671, 2)

In [81]:
gnl_p_df = innerjoin(gnl_df, p_df, on = :geonames_id);

In [82]:
Wds.filltable!("place_label", select(gnl_p_df, Not([:isShort, :isColloquial, :from, :to, :name])))

┌ Info: Rows inserted: 862
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1171


862

### Frankreich

In [261]:
cc = "FR"

"FR"

In [262]:
gnl_filename = joinpath(gnl_path, cc, cc * ".txt")

"C:\\Users\\georg\\Documents\\projekte-doc\\WiagDataSetup\\GeoNames\\alternatenames\\FR\\FR.txt"

In [263]:
gnl_df = CSV.read(gnl_filename, DataFrame, header=gnl_header, types=gnl_types);

In [264]:
size(gnl_df)

(436124, 10)

In [265]:
gnl_df = filter(:lang => filter_lang, gnl_df);

In [266]:
size(gnl_df)

(69020, 10)

Bundesländer/Kantone/Regionen einlesen, um relevante Namen zu übernehmen

In [267]:
sql = "SELECT name, geonames_id " *
      "FROM country_level_1 WHERE country_code = '" * cc * "'";
cl_df = DBInterface.execute(Wds.dbwiag, sql) |> DataFrame;

In [268]:
gnl_cl_df = innerjoin(gnl_df, cl_df, on = :geonames_id);

In [269]:
size(gnl_cl_df)

(75, 11)

In [270]:
Wds.filltable!("place_label", select(gnl_cl_df, Not([:isShort, :isColloquial, :from, :to, :name])))

┌ Info: Rows inserted: 75
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1171


75

Orte einlesen, um nur relevante Namen zu übernehmen

In [89]:
sql = "SELECT name, geonames_id " *
      "FROM place WHERE country_code = '" * cc * "'";
p_df = DBInterface.execute(Wds.dbwiag, sql) |> DataFrame;

In [90]:
size(p_df)

(80885, 2)

In [91]:
gnl_p_df = innerjoin(gnl_df, p_df, on = :geonames_id);

In [92]:
Wds.filltable!("place_label", select(gnl_p_df, Not([:isShort, :isColloquial, :from, :to, :name])))

┌ Info: 10000
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1148
┌ Info: 20000
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1148
┌ Info: 30000
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1148
┌ Info: Rows inserted: 33108
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1171


33108

### Italien

In [271]:
cc = "IT"

"IT"

In [272]:
gnl_filename = joinpath(gnl_path, cc, cc * ".txt")

"C:\\Users\\georg\\Documents\\projekte-doc\\WiagDataSetup\\GeoNames\\alternatenames\\IT\\IT.txt"

In [273]:
gnl_df = CSV.read(gnl_filename, DataFrame, header=gnl_header, types=gnl_types);

In [274]:
size(gnl_df)

(235515, 10)

In [275]:
gnl_df = filter(:lang => filter_lang, gnl_df);

In [276]:
size(gnl_df)

(55527, 10)

Bundesländer/Kantone/Regionen einlesen, um relevante Namen zu übernehmen

In [277]:
sql = "SELECT name, geonames_id " *
      "FROM country_level_1 WHERE country_code = '" * cc * "'";
cl_df = DBInterface.execute(Wds.dbwiag, sql) |> DataFrame;

In [278]:
gnl_cl_df = innerjoin(gnl_df, cl_df, on = :geonames_id);

In [279]:
size(gnl_cl_df)

(132, 11)

In [280]:
Wds.filltable!("place_label", select(gnl_cl_df, Not([:isShort, :isColloquial, :from, :to, :name])))

┌ Info: Rows inserted: 132
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1171


132

Orte einlesen, um nur relevante Namen zu übernehmen

In [99]:
sql = "SELECT name, geonames_id " *
      "FROM place WHERE country_code = '" * cc * "'";
p_df = DBInterface.execute(Wds.dbwiag, sql) |> DataFrame;

In [100]:
size(p_df)

(62844, 2)

In [101]:
gnl_p_df = innerjoin(gnl_df, p_df, on = :geonames_id);

In [102]:
Wds.filltable!("place_label", select(gnl_p_df, Not([:isShort, :isColloquial, :from, :to, :name])))

┌ Info: 10000
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1148
┌ Info: 20000
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1148
┌ Info: 30000
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1148
┌ Info: Rows inserted: 32162
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1171


32162

### Schweiz

In [281]:
cc = "CH"

"CH"

In [282]:
gnl_filename = joinpath(gnl_path, cc, cc * ".txt")

"C:\\Users\\georg\\Documents\\projekte-doc\\WiagDataSetup\\GeoNames\\alternatenames\\CH\\CH.txt"

In [283]:
gnl_df = CSV.read(gnl_filename, DataFrame, header=gnl_header, types=gnl_types);

In [284]:
size(gnl_df)

(89219, 10)

In [285]:
gnl_df = filter(:lang => filter_lang, gnl_df);

In [286]:
size(gnl_df)

(55654, 10)

Bundesländer/Kantone/Regionen einlesen, um relevante Namen zu übernehmen

In [287]:
sql = "SELECT name, geonames_id " *
      "FROM country_level_1 WHERE country_code = '" * cc * "'";
cl_df = DBInterface.execute(Wds.dbwiag, sql) |> DataFrame;

In [288]:
gnl_cl_df = innerjoin(gnl_df, cl_df, on = :geonames_id);

In [289]:
size(gnl_cl_df)

(260, 11)

In [290]:
Wds.filltable!("place_label", select(gnl_cl_df, Not([:isShort, :isColloquial, :from, :to, :name])))

┌ Info: Rows inserted: 260
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1171


260

Orte einlesen, um nur relevante Namen zu übernehmen

In [109]:
sql = "SELECT name, geonames_id " *
      "FROM place WHERE country_code = '" * cc * "'";
p_df = DBInterface.execute(Wds.dbwiag, sql) |> DataFrame;

In [110]:
size(p_df)

(13265, 2)

In [111]:
gnl_p_df = innerjoin(gnl_df, p_df, on = :geonames_id);

In [112]:
Wds.filltable!("place_label", select(gnl_p_df, Not([:isShort, :isColloquial, :from, :to, :name])))

┌ Info: Rows inserted: 6277
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1171


6277

### Österreich

In [291]:
cc = "AT"

"AT"

In [292]:
gnl_filename = joinpath(gnl_path, cc, cc * ".txt")

"C:\\Users\\georg\\Documents\\projekte-doc\\WiagDataSetup\\GeoNames\\alternatenames\\AT\\AT.txt"

In [293]:
gnl_df = CSV.read(gnl_filename, DataFrame, header=gnl_header, types=gnl_types);

In [294]:
size(gnl_df)

(30649, 10)

In [295]:
gnl_df = filter(:lang => filter_lang, gnl_df);

In [296]:
size(gnl_df)

(1820, 10)

Bundesländer/Kantone/Regionen einlesen, um relevante Namen zu übernehmen

In [297]:
sql = "SELECT name, geonames_id " *
      "FROM country_level_1 WHERE country_code = '" * cc * "'";
cl_df = DBInterface.execute(Wds.dbwiag, sql) |> DataFrame;

In [298]:
gnl_cl_df = innerjoin(gnl_df, cl_df, on = :geonames_id);

In [299]:
size(gnl_cl_df)

(56, 11)

In [300]:
Wds.filltable!("place_label", select(gnl_cl_df, Not([:isShort, :isColloquial, :from, :to, :name])))

┌ Info: Rows inserted: 56
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1171


56

Orte einlesen, um nur relevante Namen zu übernehmen

In [125]:
sql = "SELECT name, geonames_id " *
      "FROM place WHERE country_code = '" * cc * "'";
p_df = DBInterface.execute(Wds.dbwiag, sql) |> DataFrame;

In [126]:
size(p_df)

(20924, 2)

In [127]:
gnl_p_df = innerjoin(gnl_df, p_df, on = :geonames_id);

In [128]:
Wds.filltable!("place_label", select(gnl_p_df, Not([:isShort, :isColloquial, :from, :to, :name])))

┌ Info: Rows inserted: 728
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1171


728

### Dänemark

In [301]:
cc = "DK"

"DK"

In [302]:
gnl_filename = joinpath(gnl_path, cc, cc * ".txt")

"C:\\Users\\georg\\Documents\\projekte-doc\\WiagDataSetup\\GeoNames\\alternatenames\\DK\\DK.txt"

In [303]:
gnl_df = CSV.read(gnl_filename, DataFrame, header=gnl_header, types=gnl_types);

In [304]:
size(gnl_df)

(19391, 10)

In [305]:
gnl_df = filter(:lang => filter_lang, gnl_df);

In [306]:
size(gnl_df)

(1136, 10)

Bundesländer/Kantone/Regionen einlesen, um relevante Namen zu übernehmen

In [307]:
sql = "SELECT name, geonames_id " *
      "FROM country_level_1 WHERE country_code = '" * cc * "'";
cl_df = DBInterface.execute(Wds.dbwiag, sql) |> DataFrame;

In [308]:
gnl_cl_df = innerjoin(gnl_df, cl_df, on = :geonames_id);

In [309]:
size(gnl_cl_df)

(34, 11)

In [310]:
Wds.filltable!("place_label", select(gnl_cl_df, Not([:isShort, :isColloquial, :from, :to, :name])))

┌ Info: Rows inserted: 34
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1171


34

Orte einlesen, um nur relevante Namen zu übernehmen

In [135]:
sql = "SELECT name, geonames_id " *
      "FROM place WHERE country_code = '" * cc * "'";
p_df = DBInterface.execute(Wds.dbwiag, sql) |> DataFrame;

In [136]:
size(p_df)

(7261, 2)

In [137]:
gnl_p_df = innerjoin(gnl_df, p_df, on = :geonames_id);

In [138]:
Wds.filltable!("place_label", select(gnl_p_df, Not([:isShort, :isColloquial, :from, :to, :name])))

┌ Info: Rows inserted: 669
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1171


669

### Polen

In [311]:
cc = "PL"

"PL"

In [312]:
gnl_filename = joinpath(gnl_path, cc, cc * ".txt")

"C:\\Users\\georg\\Documents\\projekte-doc\\WiagDataSetup\\GeoNames\\alternatenames\\PL\\PL.txt"

In [313]:
gnl_df = CSV.read(gnl_filename, DataFrame, header=gnl_header, types=gnl_types);

In [314]:
size(gnl_df)

(113353, 10)

In [315]:
gnl_df = filter(:lang => filter_lang, gnl_df);

In [316]:
size(gnl_df)

(8635, 10)

Bundesländer/Kantone/Regionen einlesen, um relevante Namen zu übernehmen

In [317]:
sql = "SELECT name, geonames_id " *
      "FROM country_level_1 WHERE country_code = '" * cc * "'";
cl_df = DBInterface.execute(Wds.dbwiag, sql) |> DataFrame;

In [318]:
gnl_cl_df = innerjoin(gnl_df, cl_df, on = :geonames_id);

In [319]:
size(gnl_cl_df)

(130, 11)

In [320]:
Wds.filltable!("place_label", select(gnl_cl_df, Not([:isShort, :isColloquial, :from, :to, :name])))

┌ Info: Rows inserted: 130
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1171


130

Orte einlesen, um nur relevante Namen zu übernehmen

In [145]:
sql = "SELECT name, geonames_id " *
      "FROM place WHERE country_code = '" * cc * "'";
p_df = DBInterface.execute(Wds.dbwiag, sql) |> DataFrame;

In [146]:
size(p_df)

(47411, 2)

In [147]:
gnl_p_df = innerjoin(gnl_df, p_df, on = :geonames_id);

In [148]:
Wds.filltable!("place_label", select(gnl_p_df, Not([:isShort, :isColloquial, :from, :to, :name])))

┌ Info: Rows inserted: 3859
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1171


3859

### Luxemburg

In [321]:
cc = "LU"

"LU"

In [322]:
gnl_filename = joinpath(gnl_path, cc, cc * ".txt")

"C:\\Users\\georg\\Documents\\projekte-doc\\WiagDataSetup\\GeoNames\\alternatenames\\LU\\LU.txt"

In [323]:
gnl_df = CSV.read(gnl_filename, DataFrame, header=gnl_header, types=gnl_types);

In [324]:
size(gnl_df)

(7785, 10)

In [325]:
gnl_df = filter(:lang => filter_lang, gnl_df);

In [326]:
size(gnl_df)

(362, 10)

Bundesländer/Kantone/Regionen einlesen, um relevante Namen zu übernehmen

In [327]:
sql = "SELECT name, geonames_id " *
      "FROM country_level_1 WHERE country_code = '" * cc * "'";
cl_df = DBInterface.execute(Wds.dbwiag, sql) |> DataFrame;

In [328]:
gnl_cl_df = innerjoin(gnl_df, cl_df, on = :geonames_id);

In [329]:
size(gnl_cl_df)

(29, 11)

In [330]:
Wds.filltable!("place_label", select(gnl_cl_df, Not([:isShort, :isColloquial, :from, :to, :name])))

┌ Info: Rows inserted: 29
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1171


29

Orte einlesen, um nur relevante Namen zu übernehmen

In [155]:
sql = "SELECT name, geonames_id " *
      "FROM place WHERE country_code = '" * cc * "'";
p_df = DBInterface.execute(Wds.dbwiag, sql) |> DataFrame;

In [156]:
size(p_df)

(692, 2)

In [157]:
gnl_p_df = innerjoin(gnl_df, p_df, on = :geonames_id);

In [158]:
Wds.filltable!("place_label", select(gnl_p_df, Not([:isShort, :isColloquial, :from, :to, :name])))

┌ Info: Rows inserted: 203
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1171


203

### Tschechien

In [331]:
cc = "CZ"

"CZ"

In [332]:
gnl_filename = joinpath(gnl_path, cc, cc * ".txt")

"C:\\Users\\georg\\Documents\\projekte-doc\\WiagDataSetup\\GeoNames\\alternatenames\\CZ\\CZ.txt"

In [333]:
gnl_df = CSV.read(gnl_filename, DataFrame, header=gnl_header, types=gnl_types);

In [334]:
size(gnl_df)

(34389, 10)

In [335]:
gnl_df = filter(:lang => filter_lang, gnl_df);

In [336]:
size(gnl_df)

(1900, 10)

Bundesländer/Kantone/Regionen einlesen, um relevante Namen zu übernehmen

In [337]:
sql = "SELECT name, geonames_id " *
      "FROM country_level_1 WHERE country_code = '" * cc * "'";
cl_df = DBInterface.execute(Wds.dbwiag, sql) |> DataFrame;

In [338]:
gnl_cl_df = innerjoin(gnl_df, cl_df, on = :geonames_id);

In [339]:
size(gnl_cl_df)

(36, 11)

In [340]:
Wds.filltable!("place_label", select(gnl_cl_df, Not([:isShort, :isColloquial, :from, :to, :name])))

┌ Info: Rows inserted: 36
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1171


36

Orte einlesen, um nur relevante Namen zu übernehmen

In [165]:
sql = "SELECT name, geonames_id " *
      "FROM place WHERE country_code = '" * cc * "'";
p_df = DBInterface.execute(Wds.dbwiag, sql) |> DataFrame;

In [166]:
size(p_df)

(16445, 2)

In [167]:
gnl_p_df = innerjoin(gnl_df, p_df, on = :geonames_id);

In [168]:
Wds.filltable!("place_label", select(gnl_p_df, Not([:isShort, :isColloquial, :from, :to, :name])))

┌ Info: Rows inserted: 1240
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1171


1240

### Liechtenstein

In [412]:
cc = "LI"

"LI"

In [413]:
gnl_filename = joinpath(gnl_path, cc, cc * ".txt")

"C:\\Users\\georg\\Documents\\projekte-doc\\WiagDataSetup\\GeoNames\\alternatenames\\LI\\LI.txt"

In [414]:
gnl_df = CSV.read(gnl_filename, DataFrame, header=gnl_header, types=gnl_types);

In [415]:
size(gnl_df)

(906, 10)

In [416]:
gnl_df = filter(:lang => filter_lang, gnl_df);

In [417]:
size(gnl_df)

(322, 10)

Bundesländer/Kantone/Regionen einlesen, um relevante Namen zu übernehmen

In [418]:
sql = "SELECT name, geonames_id " *
      "FROM country_level_1 WHERE country_code = '" * cc * "'";
cl_df = DBInterface.execute(Wds.dbwiag, sql) |> DataFrame;

In [419]:
gnl_cl_df = innerjoin(gnl_df, cl_df, on = :geonames_id);

In [420]:
size(gnl_cl_df)

(0, 11)

In [350]:
Wds.filltable!("place_label", select(gnl_cl_df, Not([:isShort, :isColloquial, :from, :to, :name])))

┌ Info: Rows inserted: 0
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1171


0

Orte einlesen, um nur relevante Namen zu übernehmen

In [175]:
sql = "SELECT name, geonames_id " *
      "FROM place WHERE country_code = '" * cc * "'";
p_df = DBInterface.execute(Wds.dbwiag, sql) |> DataFrame;

In [176]:
size(p_df)

(361, 2)

In [177]:
gnl_p_df = innerjoin(gnl_df, p_df, on = :geonames_id);

In [178]:
Wds.filltable!("place_label", select(gnl_p_df, Not([:isShort, :isColloquial, :from, :to, :name])))

┌ Info: Rows inserted: 72
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1171


72

### Lettland (LV)

In [421]:
cc = "LV"

"LV"

In [422]:
gnl_filename = joinpath(gnl_path, cc, cc * ".txt")

"C:\\Users\\georg\\Documents\\projekte-doc\\WiagDataSetup\\GeoNames\\alternatenames\\LV\\LV.txt"

In [423]:
gnl_df = CSV.read(gnl_filename, DataFrame, header=gnl_header, types=gnl_types);

In [424]:
size(gnl_df)

(21074, 10)

In [425]:
gnl_df = filter(:lang => filter_lang, gnl_df);

In [426]:
size(gnl_df)

(764, 10)

Bundesländer/Kantone/Regionen einlesen, um relevante Namen zu übernehmen

In [427]:
sql = "SELECT name, geonames_id " *
      "FROM country_level_1 WHERE country_code = '" * cc * "'";
cl_df = DBInterface.execute(Wds.dbwiag, sql) |> DataFrame;

In [428]:
gnl_cl_df = innerjoin(gnl_df, cl_df, on = :geonames_id);

In [429]:
size(gnl_cl_df)

(29, 11)

In [430]:
Wds.filltable!("place_label", select(gnl_cl_df, Not([:isShort, :isColloquial, :from, :to, :name])))

┌ Info: Rows inserted: 29
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1171


29

Orte einlesen, um nur relevante Namen zu übernehmen

In [185]:
sql = "SELECT name, geonames_id " *
      "FROM place WHERE country_code = '" * cc * "'";
p_df = DBInterface.execute(Wds.dbwiag, sql) |> DataFrame;

In [186]:
size(p_df)

(4091, 2)

In [187]:
gnl_p_df = innerjoin(gnl_df, p_df, on = :geonames_id);

In [188]:
Wds.filltable!("place_label", select(gnl_p_df, Not([:isShort, :isColloquial, :from, :to, :name])))

┌ Info: Rows inserted: 529
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1171


529

### Littauen (LT)

In [431]:
cc = "LT"

"LT"

In [432]:
gnl_filename = joinpath(gnl_path, cc, cc * ".txt")

"C:\\Users\\georg\\Documents\\projekte-doc\\WiagDataSetup\\GeoNames\\alternatenames\\LT\\LT.txt"

In [433]:
gnl_df = CSV.read(gnl_filename, DataFrame, header=gnl_header, types=gnl_types);

In [434]:
size(gnl_df)

(51548, 10)

In [435]:
gnl_df = filter(:lang => filter_lang, gnl_df);

In [436]:
size(gnl_df)

(971, 10)

Bundesländer/Kantone/Regionen einlesen, um relevante Namen zu übernehmen

In [437]:
sql = "SELECT name, geonames_id " *
      "FROM country_level_1 WHERE country_code = '" * cc * "'";
cl_df = DBInterface.execute(Wds.dbwiag, sql) |> DataFrame;

In [438]:
gnl_cl_df = innerjoin(gnl_df, cl_df, on = :geonames_id);

In [439]:
size(gnl_cl_df)

(19, 11)

In [440]:
Wds.filltable!("place_label", select(gnl_cl_df, Not([:isShort, :isColloquial, :from, :to, :name])))

┌ Info: Rows inserted: 19
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1171


19

Orte einlesen, um nur relevante Namen zu übernehmen

In [195]:
sql = "SELECT name, geonames_id " *
      "FROM place WHERE country_code = '" * cc * "'";
p_df = DBInterface.execute(Wds.dbwiag, sql) |> DataFrame;

In [196]:
size(p_df)

(19854, 2)

In [197]:
gnl_p_df = innerjoin(gnl_df, p_df, on = :geonames_id);

In [198]:
Wds.filltable!("place_label", select(gnl_p_df, Not([:isShort, :isColloquial, :from, :to, :name])))

┌ Info: Rows inserted: 787
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1171


787

   ### Kroatien

In [441]:
cc = "HR"

"HR"

In [442]:
gnl_filename = joinpath(gnl_path, cc, cc * ".txt")

"C:\\Users\\georg\\Documents\\projekte-doc\\WiagDataSetup\\GeoNames\\alternatenames\\HR\\HR.txt"

In [443]:
gnl_df = CSV.read(gnl_filename, DataFrame, header=gnl_header, types=gnl_types);

In [444]:
size(gnl_df)

(24260, 10)

In [445]:
gnl_df = filter(:lang => filter_lang, gnl_df);

In [446]:
size(gnl_df)

(1218, 10)

Bundesländer/Kantone/Regionen einlesen, um relevante Namen zu übernehmen

In [447]:
sql = "SELECT name, geonames_id " *
      "FROM country_level_1 WHERE country_code = '" * cc * "'";
cl_df = DBInterface.execute(Wds.dbwiag, sql) |> DataFrame;

In [448]:
gnl_cl_df = innerjoin(gnl_df, cl_df, on = :geonames_id);

In [449]:
size(gnl_cl_df)

(52, 11)

In [450]:
Wds.filltable!("place_label", select(gnl_cl_df, Not([:isShort, :isColloquial, :from, :to, :name])))

┌ Info: Rows inserted: 52
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1171


52

Orte einlesen, um nur relevante Namen zu übernehmen

In [205]:
sql = "SELECT name, geonames_id " *
      "FROM place WHERE country_code = '" * cc * "'";
p_df = DBInterface.execute(Wds.dbwiag, sql) |> DataFrame;

In [206]:
size(p_df)

(9902, 2)

In [207]:
gnl_p_df = innerjoin(gnl_df, p_df, on = :geonames_id);

In [208]:
Wds.filltable!("place_label", select(gnl_p_df, Not([:isShort, :isColloquial, :from, :to, :name])))

┌ Info: Rows inserted: 481
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1171


481

### Estland

In [451]:
cc = "EE"

"EE"

In [452]:
gnl_filename = joinpath(gnl_path, cc, cc * ".txt")

"C:\\Users\\georg\\Documents\\projekte-doc\\WiagDataSetup\\GeoNames\\alternatenames\\EE\\EE.txt"

In [453]:
gnl_df = CSV.read(gnl_filename, DataFrame, header=gnl_header, types=gnl_types);

In [454]:
size(gnl_df)

(28962, 10)

In [455]:
gnl_df = filter(:lang => filter_lang, gnl_df);

In [456]:
size(gnl_df)

(978, 10)

Bundesländer/Kantone/Regionen einlesen, um relevante Namen zu übernehmen

In [457]:
sql = "SELECT name, geonames_id " *
      "FROM country_level_1 WHERE country_code = '" * cc * "'";
cl_df = DBInterface.execute(Wds.dbwiag, sql) |> DataFrame;

In [458]:
gnl_cl_df = innerjoin(gnl_df, cl_df, on = :geonames_id);

In [459]:
size(gnl_cl_df)

(66, 11)

In [460]:
Wds.filltable!("place_label", select(gnl_cl_df, Not([:isShort, :isColloquial, :from, :to, :name])))

┌ Info: Rows inserted: 66
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1171


66

Orte einlesen, um nur relevante Namen zu übernehmen

In [215]:
sql = "SELECT name, geonames_id " *
      "FROM place WHERE country_code = '" * cc * "'";
p_df = DBInterface.execute(Wds.dbwiag, sql) |> DataFrame;

In [216]:
size(p_df)

(7059, 2)

In [217]:
gnl_p_df = innerjoin(gnl_df, p_df, on = :geonames_id);

In [218]:
Wds.filltable!("place_label", select(gnl_p_df, Not([:isShort, :isColloquial, :from, :to, :name])))

┌ Info: Rows inserted: 663
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1171


663

### Rußland

In [219]:
cc = "RU"

"RU"

In [220]:
gnl_filename = joinpath(gnl_path, cc, cc * ".txt")

"C:\\Users\\georg\\Documents\\projekte-doc\\WiagDataSetup\\GeoNames\\alternatenames\\RU\\RU.txt"

In [221]:
gnl_df = CSV.read(gnl_filename, DataFrame, header=gnl_header, types=gnl_types);

In [222]:
size(gnl_df)

(1067822, 10)

In [223]:
gnl_df = filter(:lang => filter_lang, gnl_df);

In [224]:
size(gnl_df)

(46907, 10)

Orte einlesen, um nur relevante Namen zu übernehmen

In [225]:
sql = "SELECT name, geonames_id " *
      "FROM place WHERE country_code = '" * cc * "'";
p_df = DBInterface.execute(Wds.dbwiag, sql) |> DataFrame;

In [226]:
size(p_df)

(1, 2)

In [227]:
gnl_p_df = innerjoin(gnl_df, p_df, on = :geonames_id);

In [228]:
Wds.filltable!("place_label", select(gnl_p_df, Not([:isShort, :isColloquial, :from, :to, :name])))

┌ Info: Rows inserted: 11
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1171


11

### Deutsche Namen eintragen

Trage in `place` für Orte in Deutschland, Österreich und der Schweiz den deutschen Namen ein.

In [8]:
using MySQL, DataFrames

In [10]:
db_exec(sql) = DBInterface.execute(Wds.dbwiag, sql) |> DataFrame

db_exec (generic function with 1 method)

In [13]:
sql = "SELECT label, p.name FROM place_label AS pl " *
"JOIN place AS p ON pl.geonames_id = p.geonames_id " *
"WHERE pl.lang = 'de' AND p.country_code = 'DE' " *
"AND pl.label <> p.name " *
"LIMIT 12"
df_name_udt = db_exec(sql)

,label,name
,String?,String?
1,Kassel-Wahlershausen,Wahlershausen
2,Brandhofen,Spohla
3,Seebad Ahlbeck,Ahlbeck
4,Stadt Schwedt/Oder,Schwedt (Oder)
5,Dreiwerden,Schönborn-Dreiwerden
6,Groß Salze,Salzelmen
7,Bad Salzelmen,Salzelmen
8,Grenzkirch,Podrosche
9,Boltenhagen,Ostseebad Boltenhagen


In [15]:
sql = "SELECT count(*) FROM place_label AS pl " *
"JOIN place AS p ON pl.geonames_id = p.geonames_id " *
"WHERE pl.lang = 'de' AND p.country_code = 'DE' " *
"AND pl.label <> p.name "
n = db_exec(sql)

,count(*)
,Int64
1,747


Es scheint im Allgemeinen keine gute Idee zu sein, generell den deutschen Eintrag aus `place_label` zu übernehmen. Man wird einzelne Orte evtl. redaktionell bearbeiten müssen, indem man einen deutschen Namen in `place_label` als bevorzugten Namen auszeichnet.

## Quellen/Institutionen für externe IDs

Quelle: Tabelle `wiag.external_url_type`

In [9]:
using CSV, DataFrames, MySQL

In [10]:
db_exec(sql) = DBInterface.execute(Wds.dbwiag, sql)

db_exec (generic function with 1 method)

In [ ]:
sql = "SELECT id_url_type as id, url_type, url_name_formatter, url_formatter, url_value_example, display_order " *
    "FROM wiag.external_url_type"
df_authority = DBInterface.execute(Wds.dbwiag, sql) |> DataFrame;

In [ ]:
df_authority[1:5, [:id, :url_type, :url_name_formatter]]

In [ ]:
Wds.filltable!("authority", df_authority, clear_table=true);

Gemeinsamen Verbundkatalog (GVK) aufnehmen

In [ ]:
sql_gvk = "INSERT INTO authority (id, url_name_formatter, url_formatter, url_value_example)" *
    "VALUES (53, 'GVK - Gemeinsamer Verbundkatalog', 'https://kxp.k10plus.de/DB=2.1/PPNSET?PPN=', '02621539X')"

In [ ]:
DBInterface.execute(Wds.dbwiag, sql_gvk);

*TODO* evtl. `url_type` für GVK ergänzen

Personendatenbank der Germania Sacra aufnehmen

In [11]:
sql = "INSERT INTO authority (id, url_name_formatter, url_formatter, url_value_example)" *
    "VALUES (200, 'Personendatenbank der Germania Sacra', 'http://personendatenbank.germania-sacra.de/index/gsn/', '079-00974-001')"
db_exec(sql);

## Quellen/Referenzen

### Referenzen für Bischöfe und Bistümer
Quelle: Tabelle `wiag.reference`

In [ ]:
sql = "SELECT id_ref as id, " *
    "full_citation, " *
    "author_editor, " *
    "onlineressource as online_resource, " *
    "short_title as title_short, " *
    "ri_opac_id, " *
    "year_of_publication as year_publication, " *
    "isbn, " *
    "sort as display_order " *
    "FROM wiag.reference " *
    "WHERE id_ref <= 5"
df_ref_gatz = DBInterface.execute(Wds.dbwiag, sql) |> DataFrame;

In [ ]:
df_ref_gatz[:, [:id, :title_short, :isbn]]

In [ ]:
Wds.filltable!("reference_volume", df_ref_gatz)

GVK in `ref_id_external` eintragen

In [10]:
sql = "SELECT id_ref as reference_id, " *
    "53 as authority_id, " *
    "short_title as title_short, " *
    "gbv " *    
    "FROM wiag.reference " *
    "WHERE id_ref <= 5"
df_ref_ext = DBInterface.execute(Wds.dbwiag, sql) |> DataFrame;

In [11]:
filter(:gbv => !ismissing, df_ref_ext)

,reference_id,authority_id,title_short,gbv
,Int32,Int64,String?,String?
1,1,53,"Gatz, Bischöfe 1198 bis 1448",https://kxp.k10plus.de/DB=2.1/PPNSET?PPN=336676557
2,2,53,"Gatz, Bistümer",https://kxp.k10plus.de/DB=2.1/PPNSET?PPN=365121304
3,3,53,"Gatz, Bischöfe 1448 bis 1648",https://kxp.k10plus.de/DB=2.1/PPNSET?PPN=215267079
4,4,53,"Gatz, Bischöfe 1648 bis 1803",https://kxp.k10plus.de/DB=2.1/PPNSET?PPN=02621539X
5,5,53,"Gatz, Bischöfe 1785/1803 bis 1945",https://kxp.k10plus.de/DB=2.1/PPNSET?PPN=024331090


In [13]:
function extract_ppn(s)
    rgm = match(r"PPN=(.+$)", s)
    ppn = nothing
    if !isnothing(rgm)
        ppn = rgm[1]
    end
end

extract_ppn (generic function with 1 method)

In [ ]:
extract_ppn("https://kxp.k10plus.de/DB=2.1/PPNSET?PPN=215267079")

In [14]:
df_ref_ext[!, :value] .= extract_ppn.(df_ref_ext[!, :gbv])

5-element Vector{SubString{String}}:
 "336676557"
 "365121304"
 "215267079"
 "02621539X"
 "024331090"

In [ ]:
Wds.filltable!("ref_id_external", df_ref_ext[!, [:reference_id, :authority_id, :value]])

### Referenzen für Domherren der Germania Sacra
Quelle: Tabelle `gso_in_202202.books`

In [30]:
using DataFrames

In [20]:
gso_db = "gso_in_202202"
table_name = "books"
sql = "SELECT id, nummer, titel, kurztitel, autoren, jahr, `order` as gs_display_order, uri
FROM $(gso_db).$(table_name)"
df_ref_gs = Wds.sql_df(sql);

In [11]:
size(df_ref_gs, 1)

87

In [14]:
df_ref_gs[5:10, [:id, :nummer, :kurztitel, :gs_display_order]]

,id,nummer,kurztitel,gs_display_order
,Int32,String?,String,Int32?
1,6,"AF II, 1937","AF II, Bistum Bamberg",6
2,7,"AF II, 1966","AF II, Bistum Bamberg, Pfarreiorganisation",7
3,8,"AF III, 1938","AF III, Archidiakonat Xanten",8
4,9,NF 1,"NF 1, Würzburger Bischöfe bis 1254",9
5,10,NF 2,"NF 2, Cistercienserabtei Altenberg",10
6,11,NF 3,"NF 3, Schwesternhäuser Augustinerregel, Münster",11


In [15]:
item_type_canon_gs = 6

6

In [21]:
df_ref_gs[!, :item_type_id] .= item_type_canon_gs;

In [32]:
columns = [
    :titel => :full_citation,
    :kurztitel => :title_short,
    :nummer => :gs_volume_nr,
    :autoren => :author_editor,
    :jahr => :year_publication,
    :gs_display_order => :display_order,
    :uri => :gs_url,
    :id => :reference_id,
    :item_type_id => :item_type_id
]

9-element Vector{Pair{Symbol, Symbol}}:
            :titel => :full_citation
        :kurztitel => :title_short
           :nummer => :gs_volume_nr
          :autoren => :author_editor
             :jahr => :year_publication
 :gs_display_order => :display_order
              :uri => :gs_url
               :id => :reference_id
     :item_type_id => :item_type_id

In [27]:
names(df_ref_gs)

9-element Vector{String}:
 "id"
 "nummer"
 "titel"
 "kurztitel"
 "autoren"
 "jahr"
 "gs_display_order"
 "uri"
 "item_type_id"

In [33]:
table_name = "reference_volume"
Wds.filltable!(table_name, select(df_ref_gs, columns))

┌ Info: Rows inserted: 87
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1208


87

## Ämter
Quelle: Access, `GS_Domherren_DB` - `tbl_aemterliste_normiert`

*2022-01-13*

In [17]:
using MySQL, DataFrames, Dates

In [9]:
item_type_id = 8;

In [26]:
sql = "SELECT id_amt_norm AS id_in_source, AmtNorm AS name, Bemerkung AS note 
FROM domherr.tbl_aemterliste_normiert"
df_role = Wds.sql_df(sql);

Einträge in `item`

Bestehende Einträge löschen

In [13]:
table_name = "item";
sql = "DELETE FROM $(table_name) WHERE item_type_id = $(item_type_id)";
DBInterface.execute(Wds.dbwiag, sql)

MySQL.TextCursor{true}(MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag2"), "DELETE FROM item WHERE item_type_id = 8", 0, -1, 113, MySQL.API.MYSQL_RES(Ptr{Nothing} @0x0000000000000000), Symbol[], Type[], Dict{Symbol, Int64}(), 0, 1, false)

In [27]:
names(df_role)

3-element Vector{String}:
 "id_in_source"
 "name"
 "note"

In [15]:
columns = [
    :id_in_source => :id_in_source,
    :name => :item_in_source,
]

2-element Vector{Pair{Symbol, Symbol}}:
   :id => :id_in_source
 :name => :item_in_source

In [23]:
df_item = select(df_role, columns);

In [18]:
user_wiag_id = 1;
time_stamp = Dates.format(now(), dateformat"yyyy-mm-dd HH:MM");

In [24]:
insertcols!(
    df_item, 
    3,
    :item_type_id => item_type_id,
    :edit_status => "importiert",
    :created_by => user_wiag_id,
    :date_created => time_stamp,
    :changed_by => user_wiag_id,
    :date_changed => time_stamp,
    :is_online => 1
);

In [25]:
table_name = "item";
Wds.filltable!(table_name, df_item, clear_table = false)

┌ Info: Rows inserted: 152
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1206


152

IDs auslesen

In [29]:
table_name = "item"
sql = "SELECT id, id_in_source FROM $(table_name) WHERE item_type_id = $(item_type_id)"
dfx = Wds.sql_df(sql);

In [30]:
transform!(df_role, :id_in_source => ByRow(string) => :id_in_source);
df_role_db = innerjoin(dfx, df_role, on = :id_in_source);

In [32]:
columns = [:id, :note, :name]

3-element Vector{Symbol}:
 :id
 :note
 :name

In [33]:
table_name = "role"
Wds.filltable!(table_name, select(df_role_db, columns), clear_table = true)

┌ Info: Rows inserted: 152
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1206


152